In [ ]:
import json

with open('config.json', 'r') as f:
    config = json.load(f)

data_path = config['data_path']

corpus_path=data_path+'corpus.csv'
print(corpus_path)

In [20]:
from rank_bm25 import BM25Okapi
import pandas as pd
import pickle
from underthesea import word_tokenize

In [ ]:
corpus=pd.read_csv(corpus_path)
corpus.head()

In [ ]:
corpus_text=corpus['text'].to_list()
corpus_cid=corpus['cid'].to_list()
print(corpus_text[0])
print(corpus_cid[0])
# corpus_tokenization=[doc.split(" ") for doc in corpus_text]
# corpus_tokenization=[word_tokenize(doc) for doc in corpus_text]
# print(corpus_tokenization[0])

In [25]:
with open(data_path+'tokenized_corpus.pkl', 'rb') as f:
    corpus_tokenization = pickle.load(f)

In [26]:
legal_bm25 = BM25Okapi(corpus_tokenization)

In [ ]:
train_data=pd.read_csv(data_path+'train.csv')
data_records=train_data.to_dict(orient='records')

for point in data_records:
    cid_str=point['cid']
    point['cid'] = list(map(int, cid_str.strip('[]').split()))
print(data_records[0])

In [ ]:
# Khởi tạo danh sách tạm để lưu trữ kết quả
temp = []
# Số lượng điểm tối đa trước khi lưu vào file
batch_size = 10

for idx, point in enumerate(data_records):
    top_relevant = legal_bm25.get_top_n(word_tokenize(point['question']), corpus_cid, 10)
    hard_negatives = [item for item in top_relevant if item not in point['cid']]  
    curr=[point['qid']]
    curr.extend(hard_negatives)
    # Thêm question_id và hard_negatives vào danh sách tạm
    temp.append(curr)

    # Kiểm tra nếu đã đạt đến số lượng batch_size
    if (idx + 1) % batch_size == 0:
        # Ghi kết quả vào file
        with open('hard_negative.txt', 'a') as f:
            for item in temp:
                f.write(f"{item}\n")
        print(f'inserting successfully {idx} elements')
        # Xóa danh sách tạm để sẵn sàng cho lần ghi tiếp theo
        temp.clear()

# Lưu kết quả còn lại sau vòng lặp nếu có
if temp:
    with open('hard_negative.txt', 'a') as f:
        for item in temp:
            f.write(f"{item}\n")

